In [109]:
from lusidtools.jupyter_tools import toggle_code

"""Interest Rate Swap Valuation

Demonstrates pricing of an Interes Rate Swap based on a user defined Instrument.

Attributes
----------
instruments
aggregation
market data store
results store
quotes
"""

toggle_code("Hide docstring")

In [110]:
# Import LUSID
import lusid as lu
import lusid.models as lm
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Import Libraries
from datetime import datetime, timedelta, time
from IPython.display import display_html
import pytz
import pandas as pd
import matplotlib.pyplot as plt
from lusidjam.refreshing_token import RefreshingToken
import json
import os

# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:,.2f}'.format)

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
  secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = lu.utilities.ApiClientFactory(
        token=RefreshingToken(),
        api_secrets_filename = secrets_path,
        app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version)


LUSID Environment Initialised
LUSID SDK Version:  0.6.9487.0


## Interest Rate Swap Valuation 

This notebook illustrates construction of an _Interest Rate Swap_ and how its underlying defintions can be setup using the LUSID API, as well as how valuations and aggregations can be run using different endpoints. For valuations, we will be looking to use the 'Discounting' model, which uses a set of user-defined curves to discount each cash flow.

The example instrument we will be constructing is a vanilla interest rate swap with the following parameters:

    -  Coupon Rate: 2%
    -  Spread: 0bp
    -  Start Date: 01/10/2019
    -  Maturity: ~10Y (front stub)
    -  Currency: USD
    -  Payment Frequency: 6M
    -  Index: ESTRON 1D
    -  Side: Receive Fixed

### Table of contents
- 1. [Setup LUSID and required API objects](#1.-Setup-LUSID-and-required-API-objects)
    * [1.2 Interest Rate Swap Definition](#1.1-Interest-Rate-Swap-Definition)
    * [1.2 Curve and Market Data](#1.2-Curve-and-Market-Data)
    * [1.3 Setup Portfolio](#1.3-Setup-Portfolio)
    * [1.4 Add Transactions](#1.4-Add-Transactions)
- 2. [Valuation](#2.-Valuation)
    * [2.1 Configure the valuation recipe](#2.1-Configure-the-valuation-recipe)
    * [2.2 Aggregation](#2.2-Aggregation)
- 3. [Historical Cash Flows](#3.-Historical-Cash-Flows)
    * [3.1 Update Fixings for Historical Resets](#3.1-Update-Fixings-for-Historical-Resets)
    * [3.2 Create Transactions with Properties](#3.2-Create-Transactions-with-Properties)
    * [3.3 Aggregate Cash Flows](#3.3-Aggregate-Cash-Flows)

## 1. Setup LUSID and required API objects

In [111]:
# Setup the apis we'll use in this notebook:
instruments_api = api_factory.build(lu.api.InstrumentsApi)
aggregation_api = api_factory.build(lu.AggregationApi)
quotes_api = api_factory.build(lu.api.QuotesApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
complex_market_data_api = api_factory.build(lu.api.ComplexMarketDataApi)

In [112]:
def aggregation_result_to_dataframe(aggregation_results):
    return pd.DataFrame(aggregation_results, columns = ['Name', 'Effective At', 'Value'])

def df_add_utc(df):
    for column in df.columns:
        dtyp=str(df[column].dtype)
        # Add tzinfo to naive series or dt object
        if str(dtyp) == 'datetime':
            df[column] = df[column].apply(lambda d: d.replace(tzinfo=pytz.utc))
        elif str(dtyp) == 'datetime64[ns]':
            df[column] = df[column].apply(lambda d: d.tz_localize('UTC'))

In [113]:
# Set the scope of the instruments (optional)
instrument_scope = "EurOisNotebook"

### 1.1 Interest Rate Swap Definition

We can begin by setting the contract details of the swap which we will feed into LUSID. In this case we will be looking at a 10y EUR-OIS swap with a short front stub.

In [114]:
# Set swap details
trade_date = datetime(2022, 2, 10, 0, 0, tzinfo=pytz.utc)
start_date = trade_date
end_date = datetime(2032, 1, 31, 0, 0, tzinfo=pytz.utc)
instrument_ccy = "EUR"
payment_frequency = "6M"
index_tenor = "1D"
reference_index = "ESTRON"
notional = 10000000
cpn = 0.02
day_count_convention = "Actual360"
stub_type = "ShortFront"
name = "EUROIS.REC.10Y"
instrument_identifier = "EUR-OIS-SWAP001"

# Set the valuation date
effective_at = datetime(2022, 6, 10, 0, 0, tzinfo=pytz.utc)

Within LUSID, an interest rate swap will need to be defined by first creating each leg, which will depend on a [_flow convention_](https://www.lusid.com/docs/api#operation/UpsertFlowConventions), which sets the payment details. As shown below, this  is where we determine details such as the payment frequency and day count conventions. Notice that in this example we unitize the swap notional and pass a value of 1, where the actual notional will be passed on the transaction/holding level.

For the floating leg, we will also use a model that sets the [_index convention_](https://www.lusid.com/docs/api#operation/UpsertIndexConvention), which is where we will specify details relating to the ESTRON reference index.

In [115]:
# Create the fixed leg details
fixed_flows = lm.FlowConventions(
    currency=instrument_ccy,
    payment_frequency=payment_frequency,
    day_count_convention=day_count_convention,
    roll_convention="ModifiedFollowing",
    payment_calendars=[],
    reset_calendars=[],
    settle_days=0,
    reset_days=0
)

fixed_leg_definition = lm.LegDefinition(
    rate_or_spread=cpn,
    pay_receive="Receive",
    conventions=lm.FlowConventions(
        currency=instrument_ccy,
        payment_frequency=payment_frequency,
        day_count_convention=day_count_convention,
        roll_convention="ModifiedFollowing",
        payment_calendars=[],
        reset_calendars=[],
        settle_days=0,
        reset_days=0
    ),
    stub_type=stub_type,
    notional_exchange_type="None"
)

# Persist the fixed leg
fixed_leg = lm.FixedLeg(
    start_date=start_date,
    maturity_date=end_date,
    # Unitized notional, could also be passed directly
    notional=1,
    leg_definition=fixed_leg_definition,
    instrument_type="FixedLeg"
)

In [116]:
# Create the floating leg details
floating_leg_definition = lm.LegDefinition(
    rate_or_spread = 0,
    index_convention = lm.IndexConvention(
        currency=instrument_ccy,
        payment_tenor=index_tenor,
        fixing_reference=reference_index,
        publication_day_lag=0,
        day_count_convention=day_count_convention,
        index_name=reference_index
    ),
    pay_receive="Pay",
    conventions=lm.FlowConventions(
        currency=instrument_ccy,
        payment_frequency=payment_frequency,
        day_count_convention=day_count_convention,
        roll_convention="ModifiedFollowing",
        payment_calendars=[],
        reset_calendars=[],
        settle_days=0,
        reset_days=0
    ),
    stub_type=stub_type,
    notional_exchange_type="None",
    reset_convention="InArrears",
    compounding=lm.Compounding(
        compounding_method="Compounded",
        reset_frequency="1D",
        spread_compounding_method="SpreadExclusive"
    )
)

# Persist floating leg
floating_leg = lm.FloatingLeg(
    start_date=start_date,
    maturity_date=end_date,
    # Unitized notional, could also be passed directly
    notional=1,
    leg_definition=floating_leg_definition,
    instrument_type="FloatingLeg"
)

Using the above definitions we can move to building the swap and passing its component legs.

In [117]:
# Store legs in a list
irs_legs = [
    fixed_leg,
    floating_leg
]

#create the swap
instrument_definition = lm.InterestRateSwap(
    start_date=start_date,
    maturity_date=end_date,
    legs=irs_legs,
    instrument_type="InterestRateSwap"
)


# create instrument definition
swap_instrument_definition = lm.InstrumentDefinition(
    name=name,
    identifiers={"ClientInternal": lm.InstrumentIdValue(value=instrument_identifier)},
    definition=instrument_definition
)
upsert_request = {instrument_identifier: swap_instrument_definition}

#upsert instrument to LUSID
response = instruments_api.upsert_instruments(
    request_body=upsert_request,
    scope=instrument_scope
)

if len(response.failed) == 0:
    print(f"Instrument {swap_instrument_definition.name} was successfully upserted into LUSID")
else:
    print("An error occurred with the above upsert_instruments call, see error message:", response.failed)

Instrument EUROIS.REC.10Y was successfully upserted into LUSID


In [118]:
def get_instrument(
        identifier,
        scope,
   ):
    response = instruments_api.get_instrument(
        identifier_type="ClientInternal",
        identifier=identifier,
        scope=scope
    )
    return response

# Get and store the instrument response
response = get_instrument(instrument_identifier, instrument_scope)

# Store response leg definitions as dataframes
flt_leg_df = lusid_response_to_data_frame(response.instrument_definition.legs[1])
fix_leg_df = lusid_response_to_data_frame(response.instrument_definition.legs[0])

# Display the leg dataframes
df1_styler = flt_leg_df.style.set_table_attributes("style='display:inline'").set_caption('Floating Leg')
df2_styler = fix_leg_df.style.set_table_attributes("style='display:inline'").set_caption('Fixed Leg')
display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

,response_values
start_date,2022-02-10 00:00:00+00:00
maturity_date,2032-01-31 00:00:00+00:00
leg_definition.convention_name,None
leg_definition.conventions.currency,EUR
leg_definition.conventions.payment_frequency,6M
leg_definition.conventions.day_count_convention,Actual360
leg_definition.conventions.roll_convention,ModifiedFollowing
leg_definition.conventions.payment_calendars,[]
leg_definition.conventions.reset_calendars,[]
leg_definition.conventions.settle_days,0


### 1.2 Setting up Market Data: Curves and Resets

In order to value the swap we will need to create a curve by which we can determine the present values of the cash flows, as well as build out a forward curve for our floating leg. In this example we are creating a discount curve using EUR OIS discount factors, and for the forward curve we use a 6M FRA curve also building on discount factors. 

In [119]:
# scope used to store our market data
market_data_scope = "FBN-Market-Data"
# the market data supplier
market_supplier = "Lusid"


def upsert_discount_factor_curve(scope, effective_at, market_asset, dates, discount_factors):

    # provide the structured data file source and it's document format
    complex_market_data = lm.DiscountFactorCurveData(
        base_date=effective_at,
        dates=dates,
        discount_factors=discount_factors,
        market_data_type="DiscountFactorCurveData",
    )

    # create a unique identifier for our OIS yield curves
    complex_id = lm.ComplexMarketDataId(
        provider="Lusid",
        price_source="Lusid",
        effective_at=effective_at,
        market_asset=market_asset,
    )

    upsert_request = lm.UpsertComplexMarketDataRequest(
        market_data_id=complex_id, market_data=complex_market_data
    )

    # https://www.lusid.com/docs/api#operation/UpsertComplexMarketData
    response = complex_market_data_api.upsert_complex_market_data(
        scope=scope, request_body={market_asset: upsert_request}
    )

    if len(response.failed) > 0:
        raise Exception(f"Failed to upload yield curve {response.failed}")

    print(f"{market_asset} yield curve uploaded into scope={scope}")

EUR/EUROIS yield curve uploaded into scope=FBN-Market-Data
EUR/1D/ESTRON yield curve uploaded into scope=FBN-Market-Data


In [ ]:
# Set the discount curve
dates = [
    datetime(2022, 7, 10, tzinfo=pytz.utc),
    datetime(2022, 12, 10, tzinfo=pytz.utc),
    datetime(2023, 6, 10, tzinfo=pytz.utc),
    datetime(2025, 6, 10, tzinfo=pytz.utc),
    datetime(2032, 6, 10, tzinfo=pytz.utc)
]

discount_factors = [
    1.000511,
    0.999376,
    0.991461,
    0.947892,
    0.799767
]

# Upsert discount and forward curves
for mkt_asset in ["EUR/EUROIS", "EUR/1D/ESTRON"]:

    upsert_discount_factor_curve(
        scope=market_data_scope,
        effective_at=effective_at,
        market_asset=mkt_asset,
        dates=dates,
        discount_factors=discount_factors
    )

In [121]:
# Read fixings for ESTR
resets_df = pd.read_csv("data/eur_ois_resets.csv")
resets_df["Date"] = pd.to_datetime(resets_df["Date"],dayfirst=True)
df_add_utc(resets_df)
resets_df.head()

,Date,Rate,Index
0,2022-01-03 00:00:00+00:00,-0.58,ESTRON
1,2022-01-04 00:00:00+00:00,-0.58,ESTRON
2,2022-01-05 00:00:00+00:00,-0.58,ESTRON
3,2022-01-06 00:00:00+00:00,-0.58,ESTRON
4,2022-01-07 00:00:00+00:00,-0.58,ESTRON


In [122]:
# Create quotes request
instrument_quotes = {
            index: lm.UpsertQuoteRequest(
            quote_id=lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=row["Index"],
                    instrument_id_type="RIC",
                    quote_type="Rate",
                    field="mid",
                ),
                effective_at=row["Date"],
            ),
            metric_value=lm.MetricValue(value=row["Rate"], unit="rate"),
            scale_factor=100
        )
    for index, row in resets_df.iterrows()
}

# Upsert quotes into LUSID
response = quotes_api.upsert_quotes(
    scope=market_data_scope, request_body=instrument_quotes
)

if response.failed == {}:
    print(f"Quotes successfully loaded into LUSID. {len(response.values)} quotes loaded.")
else:
    print(f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID.")

Quotes successfully loaded into LUSID. 119 quotes loaded.


The next piece of market data necessary will be the floating leg fixings, which in this case is Euribor 6M data for the historic reset dates. For valuation effective today, we would at least require a fixing for the swap's inception date and another for the valuation data (latest fixing). 

Below we will add market data fixings for the other historical resets as well, as these will be required at a later stage when we look at historical cashflows.

### 1.3 Setup Portfolio

In order to value the swap, and understand how it impacts our cash position, we will setup a portfolio with a position in the previously defined instrument. We begin by defining the scope for our swap book, and then follow by booking a transaction to the portfolio. 

In [123]:
# Setup scope and code for the portfolio
portfolio_scope = "Finbourne-Examples"
portfolio_code = "EUROIS-IRS-Example"


try:
    transaction_portfolios_api.create_portfolio(
        scope=portfolio_scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="EUR",
            created="2010-01-01",
            sub_holding_keys=[],
            instrument_scopes=[instrument_scope]
        ),
    )
    print(f"Portfolio for {portfolio_scope} | {portfolio_code} created succesfully.")

except lu.ApiException as e:
    print(json.loads(e.body)["title"])


Portfolio for Finbourne-Examples | EUROIS-IRS-Example created succesfully.


### 1.4 Add Transactions

With the portfolio in LUSID, we can add a transaction entering a position in the swap. For this we use the `TransactionPortfoliosApi`, which is the required API to interact with a [Transaction Porftolio](#https://support.finbourne.com/what-is-a-transaction-portfolio).

This is where we will set the notional of the swap to EUR 10MM, as we unitized the swap instrument in the earlier step. This is a choice, as the alternative would be to book the notinoal at the instrument level and setting the `units` to 1 at the holding/transaction level.  However, unitizing the instrument gives further flexibility for partial wind downs during the instrument's life cycle.

In [124]:
# Book a buy transaction against the swap instrument
open_position = lm.TransactionRequest(
    transaction_id="TXN001",
    type="Buy",
    instrument_identifiers={
        "Instrument/default/ClientInternal":f"{instrument_identifier}"
    },
    transaction_date=trade_date,
    settlement_date=trade_date,
    units=notional,
    transaction_price=lm.TransactionPrice(price=1,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=0,currency="EUR"),
    exchange_rate=1,
    transaction_currency="EUR"
)

response = transaction_portfolios_api.upsert_transactions(
    scope=portfolio_scope,
    code=portfolio_code,
    transaction_request=[
        open_position,
    ]
)

print(f"Transaction updated for effective time: {response.version.effective_from}.")

Transaction updated for effective time: 2022-02-10 00:00:00+00:00.


## 2. Valuation

Following the initial setup, we can see to configure how LUSID will conduct valuation on the swap. This introduces the concept of recipes, which are a set of steps we specify to the valuation engine.

### 2.1 Configure the valuation recipe

We begin by defining the pricing context under which valuation will be done, this allows to select the model under which we can value the instrument. In our case we have selected _"Discounting"_, which will calcualte the swap's present value using a discounted cash flow model. 

In [125]:
#Set recipe code
recipe_code = "EurOisValuation"

# Populate recipe parameters
configuration_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    market=lm.MarketContext(
        market_rules=[

            lm.MarketDataKeyRule(
                key="Quote.RIC.*",
                supplier="Lusid",
                data_scope=market_data_scope,
                quote_type="Rate",
                field="mid",
                quote_interval="5D",
            ),
            lm.MarketDataKeyRule(
                key="Rates.*.*",
                supplier="Lusid",
                data_scope=market_data_scope,
                price_source="Lusid",
                quote_type="Rate",
                field="mid",
                quote_interval="5D",
            ),
            lm.MarketDataKeyRule(
                key="Rates.*.*.*",
                supplier="Lusid",
                data_scope=market_data_scope,
                price_source="Lusid",
                quote_type="Rate",
                field="mid",
                quote_interval="5D",
            ),
        ],
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="Discounting",
                instrument_type="InterestRateSwap",
                parameters="{}",
            )
        ],
    ),
)

response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=lm.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

print(f"Configuration recipe loaded into LUSID at time {response.value}.")

Configuration recipe loaded into LUSID at time 2022-06-21 17:48:48.877153+00:00.


### 2.2 Aggregation

With the recipes and configurations set for our swap book, we can now setup a request to the valuation engine and create and return the aggregated data. 

In [127]:
# Method to query daily valuations
def get_daily_valuation(date, portfolio_scope, portfolio_code, recipe_scope, recipe_code, metrics, group_by=["Instrument/default/Name"]):

    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=recipe_scope, code=recipe_code),
        metrics=metrics,
        group_by=group_by,
        portfolio_entity_ids=[
            lm.PortfolioEntityId(scope=portfolio_scope, code=portfolio_code)
        ],
        valuation_schedule=lm.ValuationSchedule(effective_at=date),
    )

    val_data = aggregation_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)

    return vals_df


In [131]:
# Set the queryable metrics and operation to be applied on the group_by level
metrics = [
    lm.AggregateSpec("Instrument/default/Name", "Value"),
    lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
    lm.AggregateSpec("Valuation/PvInPortfolioCcy", "Value"),
    lm.AggregateSpec("Valuation/Accrued", "Value"),
    lm.AggregateSpec("Holding/default/Units", "Value"),
        ]

df = get_daily_valuation(
    date=effective_at,
    portfolio_scope=portfolio_scope,
    portfolio_code=portfolio_code,
    recipe_scope=market_data_scope,
    recipe_code=recipe_code,
    metrics=metrics
)

df

,Instrument/default/Name,Instrument/default/ClientInternal,Valuation/PvInPortfolioCcy,Valuation/Accrued,Holding/default/Units
0,EUROIS.REC.10Y,EUR-OIS-SWAP001,"-78,846.86","86,047.87","10,000,000.00"


We can now visualize the upcoming cash flows for both legs from the response, and plot the net cash flows. 

In [135]:
payment_diary = instruments_api.get_instrument_payment_diary(
    identifier_type="ClientInternal",
    identifier=instrument_identifier,
    recipe_scope=market_data_scope,
    recipe_code=recipe_code,
    effective_at=effective_at,
    scope=instrument_scope
)

payment_diary

{'href': 'https://demosetup.lusid.com/api/api/instruments/ClientInternal/EUR-OIS-SWAP001/paymentdiary?scope=EurOisNotebook&effectiveAt=2022-06-10T00%3A00%3A00.0000000%2B00%3A00&asAt=2022-06-22T11%3A03%3A55.8383800%2B00%3A00&recipeIdCode=EurOisValuation&recipeIdScope=FBN-Market-Data',
 'instrument_id': 'EUR-OIS-SWAP001',
 'instrument_id_type': 'ClientInternal',
 'instrument_scope': 'EurOisNotebook',
 'legs': [{'leg_id': 'Receive',
           'rows': [{'accrual_end': datetime.datetime(2022, 7, 29, 0, 0, tzinfo=tzutc()),
                     'accrual_start': datetime.datetime(2022, 2, 10, 0, 0, tzinfo=tzutc()),
                     'cash_flow_type': 'Coupon',
                     'currency': 'EUR',
                     'day_count_fraction': 0.46301369863013697,
                     'discount_factor': 1.00036998221896,
                     'discounted_expected_cash_flow_amount': 0.009392362610833569,
                     'forward_rate': 0.46301369863013697,
                     'is_cash_fl

In [138]:
portfolio_cash_flows = transaction_portfolios_api.get_portfolio_cash_flows(
    scope=portfolio_scope,
    code=portfolio_code,
    effective_at=effective_at,
    window_start=effective_at,
    window_end=end_date,
    recipe_id_scope=market_data_scope,
    recipe_id_code=recipe_code
)

portfolio_cash_flows

{'href': 'https://demosetup.lusid.com/api/api/transactionportfolios/Finbourne-Examples/EUROIS-IRS-Example/cashflows/06%2F10%2F2022%2000%3A00%3A00%20%2B00%3A0001%2F31%2F2032%2000%3A00%3A00%20%2B00%3A00?recipeIdCode=EurOisValuation&recipeIdScope=FBN-Market-Data&windowStart=2022-06-10T00%3A00%3A00%2B00%3A00&windowEnd=2032-01-31T00%3A00%3A00%2B00%3A00',
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://demosetup.lusid.com/app/insights/logs/0HMIK6D0OO9GG:00000032',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'next_page': None,
 'previous_page': None,
 'values': [{'amount': 93888.88888888889,
             'currency': 'EUR',
             'diagnostics': {'AccrualEnd': '07/29/2022 00:00:00 +00:00',
                             'AccrualStart': '02/10/2022 00:00:00 +00:00',
                             'CashFlowType': 'Coupon',
                            

In [139]:
portfolio_cash_flows.values

[{'amount': 93888.88888888889,
  'currency': 'EUR',
  'diagnostics': {'AccrualEnd': '07/29/2022 00:00:00 +00:00',
                  'AccrualStart': '02/10/2022 00:00:00 +00:00',
                  'CashFlowType': 'Coupon',
                  'DayCountFraction': '0.463013698630136986301369863',
                  'IsResetAvailable': 'False',
                  'PayReceive': 'Receive',
                  'PaymentDateDiscountFactor': '1.00036998221896'},
  'links': None,
  'payment_date': datetime.datetime(2022, 7, 29, 0, 0, tzinfo=tzutc()),
  'source_instrument_id': 'LUID_00003DGA',
  'source_instrument_scope': 'EurOisNotebook',
  'source_portfolio_id': {'code': 'EUROIS-IRS-Example',
                          'scope': 'Finbourne-Examples'},
  'source_transaction_id': 'TXN001'},
 {'amount': 103333.33333333333,
  'currency': 'EUR',
  'diagnostics': {'AccrualEnd': '01/31/2023 00:00:00 +00:00',
                  'AccrualStart': '07/29/2022 00:00:00 +00:00',
                  'CashFlowType': 'Coup

In [154]:
recs = {}
pays = {}

for element in portfolio_cash_flows.values:
    #print(element.diagnostics["PayReceive"])
    if element.diagnostics["PayReceive"] == "Receive":
        recs["Receive-" + str(element.payment_date)] = element
    else:
        pays["Pay-" + str(element.payment_date)] = element

combined = {**recs, **pays}
combined


{'Receive-2022-07-29 00:00:00+00:00': {'amount': 93888.88888888889,
  'currency': 'EUR',
  'diagnostics': {'AccrualEnd': '07/29/2022 00:00:00 +00:00',
                  'AccrualStart': '02/10/2022 00:00:00 +00:00',
                  'CashFlowType': 'Coupon',
                  'DayCountFraction': '0.463013698630136986301369863',
                  'IsResetAvailable': 'False',
                  'PayReceive': 'Receive',
                  'PaymentDateDiscountFactor': '1.00036998221896'},
  'links': None,
  'payment_date': datetime.datetime(2022, 7, 29, 0, 0, tzinfo=tzutc()),
  'source_instrument_id': 'LUID_00003DGA',
  'source_instrument_scope': 'EurOisNotebook',
  'source_portfolio_id': {'code': 'EUROIS-IRS-Example',
                          'scope': 'Finbourne-Examples'},
  'source_transaction_id': 'TXN001'},
 'Receive-2023-01-31 00:00:00+00:00': {'amount': 103333.33333333333,
  'currency': 'EUR',
  'diagnostics': {'AccrualEnd': '01/31/2023 00:00:00 +00:00',
                  'AccrualSta

In [ ]:
# # We can build a cash flow table from the 'cashflows' of the response
# cashflows = results.data[0]['Analytic/default/Cashflows']
#
# labels = cashflows['slices']['Pay']['labelsY']
# cf_pay = cashflows['slices']['Pay']['values']
# cf_rec = cashflows['slices']['Receive']['values']
#
# data ={
#     'Dates':[keys for keys, values in labels.items()],
#     'CF Pay':[values for keys, values in cf_pay.items()],
#     'CF Receive':[values for keys, values in cf_rec.items()],
# }
#
# # Create a dataframe and add a 'Net' column
# cf_table = pd.DataFrame(data)
# cf_table['CF Net'] = cf_table['CF Pay']+cf_table['CF Receive']
#
# cf_table.plot.bar(x='Dates', y='CF Net', color = '#ff5200')
# plt.title('Cash Flow Chart (Net)')
# plt.ylabel('EUR')
# display(cf_table)
# plt.show()

Next, we will query the payment dates from our swap instrument, which can be done with a similar aggregation as used previously for valuation purposes. However, in this case we will only query _HoldingCashFlows_, as we will need to use a historical effective date which wouldn't work when calling _valuation_ as we are missing the historical curve data. (Naturally, with historical curve data in LUSID, we could query using the same function as above.)

### 3.2 Create Transactions with Properties

Using our EUR006M fixings, we can determine the historical cash flows that would have taken place prior to today's valuation. Within LUSID, these can be stored using separate holding keys, which are a special type of transaction property that allow a user to differentiate between transactions on the same instrument. You can learn more about these type of properties [here](https://support.finbourne.com/how-are-subholding-keys-used-to-capture-different-investment-strategies). 

In our example the property will allow us to distinguish between cash flows that come from the pay/receive legs. Naturally, these could be applied using different property definitions as well, such as relating to a particular broker, custodian, fund accountant, etc.   


### 3.3 Aggregate Cash Flows

With the transactions updated into the portfolio, we can now query the holdings which will include our updated total cash balance. Given the negative rates on the floating leg, we can see that the original EUR 1MM is reduced significantly as a result of this. In order to run the query, we use the [**get_holdings**](https://www.lusid.com/docs/api/#operation/GetHoldings) call with a reference to our portfolio's scope and code. 

Using the previously set transaction properties, we can now query these to distinguish between different types of cash inflows and outfows. We should now be able to separate pay/receive-leg cash flows. Without the transaction key-value pairs, this might be difficult in a larger book of swaps, given that the receiving cash flows are negative.

Using the same API, we can run the [**get_transactions**](https://www.lusid.com/docs/api/#operation/GetTransactions) call for the scope of our portfolio. Additionally, since we are looking to separate based on values of the transaction property we can add a property filter to the call which results in the following to datasets.